In [ ]:
%cd ..

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import struct

from evgena.utils.large_files import file_sha256

In [ ]:
def read_header(fp):
    # read header
    magic, type_code, dim_count = struct.unpack('>HBB', fp.read(4))

    # check magic
    if magic != 0:
        raise ValueError('invalid magic number; expected: 0, found: {!r}'.format(magic))

    # construct shape from header
    shape = struct.unpack('>' + 'i' * dim_count, fp.read(4 * dim_count))

    return magic, type_code, dim_count, shape

def type_code_to_dtype(type_code: int) -> np.dtype:
    if type_code == 0x08:
        return np.dtype('B')
    elif type_code == 0x09:
        return np.dtype('b')
    elif type_code == 0x0B:
        return np.dtype('>i2')
    elif type_code == 0x0C:
        return np.dtype('>i4')
    elif type_code == 0x0D:
        return np.dtype('>f4')
    elif type_code == 0x0E:
        return np.dtype('>f8')
    else:
        raise ValueError('invalid type code; found {!r}'.format(type_code))
        
def to_ndarray(path):
    with open(path, 'rb') as file:
        # read in fixed part of header
        magic, type_code, dim_count, shape = read_header(file)

        # load data, reshape, change endianess
        array = np.fromfile(file, dtype=type_code_to_dtype(type_code))
        array = np.reshape(array, shape)
        return array.byteswap(True)

## Initialization
- define `dataset_source_dir` - the directory with standart MNIST formatted dataset
- define `dataset_target_path` - path where to store converted dataset .npz
- define `synset` - array when indexed with labels yielding corresponding names
- optionally define metadata - license, source etc.

In [ ]:
dataset_source_dir = 'datasets/mnist/'
dataset_target_path = 'datasets/mnist.npz'

synset = np.array([
    '0',
    '1',
    '2',
    '3',
    '4',
    '5',
    '6',
    '7',
    '8',
    '9'
])

metadata = {
    'source': np.array(['http://yann.lecun.com/exdb/mnist/'])
}

## Load MNIST like dataset

In [ ]:
print('Converting MNIST like dataset from {source} to {target}'.format(
    source=dataset_target_path, target=dataset_target_path
))

train_X_path = dataset_source_dir + 'train-images-idx3-ubyte'
train_y_path = dataset_source_dir + 'train-labels-idx1-ubyte'
test_X_path = dataset_source_dir + 't10k-images-idx3-ubyte'
test_y_path = dataset_source_dir + 't10k-labels-idx1-ubyte'

print('\tloading train_X from: {}'.format(train_X_path), end='')
train_X = to_ndarray(train_X_path)
print('; shape: {}'.format(train_X.shape))
print('\tloading train_X from: {}'.format(train_y_path), end='')
train_y = to_ndarray(train_y_path)
print('; shape: {}'.format(train_y.shape))
print('\tloading train_X from: {}'.format(test_X_path), end='')
test_X = to_ndarray(test_X_path)
print('; shape: {}'.format(test_X.shape))
print('\tloading train_X from: {}'.format(test_y_path), end='')
test_y = to_ndarray(test_y_path)
print('; shape: {}'.format(test_y.shape))

assert train_X.shape[1:] == test_X.shape[1:],\
    'Example shape mismatch between train and test set'

print('Normalizing images to [0.0,1.0] float32')
train_X = train_X.astype('f4') / 255.
test_X = test_X.astype('f4') / 255.

## Save NPrecord dataset
- at this point you can optionally modify loaded dataset - change shapes, restructure splits etc.

In [ ]:
example_type = np.dtype([('X', np.float32, train_X.shape[1:]), ('y', np.uint8)])

train = np.rec.fromarrays((train_X, train_y), dtype=example_type)
test = np.rec.fromarrays((test_X, test_y), dtype=example_type)

print('Writing dataset to {}'.format(dataset_target_path))
np.savez(dataset_target_path, train=train, test=test, synset=synset, **metadata)

print('\nDataset checksum: {}'.format(file_sha256(dataset_target_path)))

## Test converted dataset

In [ ]:
dataset = np.load(dataset_target_path)
load_train = dataset['train'].view(np.recarray)
load_test = dataset['test'].view(np.recarray)
load_synset = dataset['synset']

In [ ]:
index = 0

fig, ax = plt.subplots(ncols=2, figsize=(10,6))
fig.tight_layout()

ax[0].set_title('Label: {l} - {s}'.format(l=train.y[index], s=synset[train.y[index]]))
ax[0].imshow(train.X[index], cmap='gray')

ax[1].set_title('Label: {l} - {s}'.format(l=load_train.y[index], s=synset[load_train.y[index]]))
ax[1].imshow(load_train.X[index], cmap='gray')